## Setup
Let's install needed components. That will download a few packages needed for this exercise. 

In [ ]:
pip install --default-timeout=1000 atscale prophet

In [ ]:
from atscale.client import Client
from atscale.data_model import DataModel
from atscale.project import Project
from prophet import Prophet
import pandas as pd

## Connection

Specify connection details of your Adaptive Analytics instance. 
Once you're asked for the organization respond in the dialog box with "1".
And then please enter your password from the AtScale instance.

In [ ]:
client = Client(server='http://adaptivedemo', username='sample')
client.connect()

Select your project from the list of projects on the server. The answer should be the integer ID of the project.
And then data model is selected automatically if it's the only one.

In [ ]:
project = client.select_project()   
data_model = project.select_data_model()

## Explore your dataset
There are a number of methods prepared by AtScale in this library. They allow to explore data catalog that you have, query data, and even ingest some data back.
Please refer to this page for complete [API Reference](https://documentation.ai-link.atscale.com/user-guide/api/data_model/index.html).

In [ ]:
data_model.get_features()

In [ ]:
data_model.get_all_categorical_feature_names()

In [ ]:
data_model.get_all_numeric_feature_names()

In [ ]:
df = data_model.get_data(feature_list = ['Country','Region','m_AmountOfSale_sum'])
df = df.sort_values(by='m_AmountOfSale_sum')
df.head()

Let's get some dataset and show it on the graph

In [ ]:
import matplotlib.pyplot as plt

# We're taking sales for each date
dataframe = data_model.get_data(feature_list = ['Date','m_AmountOfSale_sum'])

# Create a line chart
plt.plot(dataframe['Date'], dataframe['m_AmountOfSale_sum'])

# Add labels and a title
plt.xlabel('Days')
plt.ylabel('Sales')
plt.title('Daily Sales Data')

# Display the chart
plt.show()

## Prediction
Let's do some simple prediction base on our semantic model

In [ ]:
# Load the historical data to train the model
data_train = data_model.get_data(
    feature_list = ['Date','m_AmountOfSale_sum'],
    filter_less = {'Date':'2021-01-01'}
    )
data_test = data_model.get_data(
    feature_list = ['Date','m_AmountOfSale_sum'],
    filter_greater = {'Date':'2021-01-01'}
    )

# For the tool we've chosen to do the prediction 'Prophet', we'll need to specify 2 columns: 'ds' and 'y'
data_train['ds'] = pd.to_datetime(data_train['Date'])
data_train.rename(columns={'m_AmountOfSale_sum': 'y'}, inplace=True)
data_test['ds'] = pd.to_datetime(data_test['Date'])
data_test.rename(columns={'m_AmountOfSale_sum': 'y'}, inplace=True)

# Initialize and fit the Prophet model
model = Prophet()
model.fit(data_train)

# Create a future dataframe for forecasting
future = pd.DataFrame()
future['ds'] = pd.date_range(start='2021-01-01', end='2021-12-31', freq='D')

# Make predictions
forecast = model.predict(future)
fig = model.plot(forecast)
fig.show()

## Writeback

Then we can put back our prediction to the data warehouse and add an aggregate to our semantic model to reflect it for other consumers.
The prediction would be available through any other BI tool for BI analysts and business users.

#### Example
This is given as example.
Next block of code will not be executed in our demo unfortunately as we have a multitenant demo environment.
But feel free to experiment with you instance of AtScale.

In [ ]:

from atscale.db.connections import Iris
db = Iris(
    username,
    host,
    namespace,
    driver,
    schema, 
    port=1972,
    password=None, 
    warehouse_id=None
    )

data_model.writeback(dbconn=db,
                    table_name= 'SalesPrediction',
                    DataFrame = forecast)

data_model.create_aggregate_feature(dataset_name='SalesPrediction',
                                    column_name='SalesForecasted',
                                    name='sum_sales_forecasted',
                                    aggregation_type='SUM')




# Fin